In [2]:

#Chandra Kandiah , Project II

import datetime
import numpy as np
import pandas as pd
from pandas_datareader import data
from pandas_datareader import DataReader
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG

backtesting.set_bokeh_output(notebook=False)

from dotenv import load_dotenv
import os 
load_dotenv( "L:/Bootcamp Work/GitHub/chandra.env")



C:\Users\chand\anaconda3\envs\pyvizenv\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

True

In [3]:
# ma_cross.py

class MarketOnClosePortfolio():
    #class MarketOnClosePortfolio(Portfolio):
       def __init__(self, symbol, bars, signals, initial_capital=100000.0):
        self.symbol = symbol        
        self.bars = bars
        self.signals = signals
        self.initial_capital = float(initial_capital)
        self.positions = self.generate_positions()
       # print (self.positions)
        
    def generate_positions(self):
        positions = pd.DataFrame(index=signals.index).fillna(0.0)
        positions[self.symbol] = 100*signals['signal']   # This strategy buys 100 shares
        return positions
       # Print(positions)
                    
    def backtest_portfolio(self):
        portfolio = self.positions*self.bars['close']
        pos_diff = self.positions.diff()

        portfolio['holdings'] = (self.positions*self.bars['close']).sum(axis=1)
        portfolio['cash'] = self.initial_capital - (pos_diff*self.bars['close']).sum(axis=1).cumsum()

        portfolio['total'] = portfolio['cash'] + portfolio['holdings']
        portfolio['returns'] = portfolio['total'].pct_change()
      
    
        return portfolio
       

In [4]:
# ma_cross.py

class MovingAverageCrossStrategy():
    
    def __init__(self, symbol, bars, short_window, long_window):
        self.symbol = symbol
        self.bars = bars

        self.short_window = short_window
        self.long_window = long_window
   
               

    def generate_signals(self):
   
       # print(bars)
        signals = pd.DataFrame(index=self.bars.index)
        signals['signal'] = 0.0

        signals['short_mavg'] = self.bars['close'].rolling(window=self.short_window, min_periods=1, center=False).mean()
        signals['long_mavg'] =  self.bars['close'].rolling(window=self.long_window, min_periods=1, center=False).mean()

        # Create a 'signal' (invested or not invested) when the short moving average crosses the long
        # moving average, but only for the period greater than the shortest moving average window
        signals['signal'][self.short_window:] = np.where(signals['short_mavg'][self.short_window:] 
            > signals['long_mavg'][self.short_window:], 1.0, 0.0)   

        # Take the difference of the signals in order to generate actual trading orders
        signals['positions'] = signals['signal'].diff()   

        return signals


In [1]:
# ma_cross.py
import os 
load_dotenv( "L:/Bootcamp Work/GitHub/chandra.env")
start_date = ('2019-1-1')
end_date = ('2019-12-31')


if __name__ == "__main__":

  
    
    symbol = ['GOOG', "MSFT", 'TSLA']
    bars= data.DataReader(symbol,'tiingo',start_date,end_date, api_key="2753db025119259790b492db79641412faa9253f")
   

    mac = MovingAverageCrossStrategy(symbol, bars, 100, 400)
    signals = mac.generate_signals()
   

    # Create a portfolio of AAPL, with $100,000 initial capital
    portfolio = MarketOnClosePortfolio(symbol, bars, signals, initial_capital=100000.0)
    #print (signals)
    
    returns = portfolio.backtest_portfolio()

     # Plot two charts to assess trades and equity curve
    fig = plt.figure()
    fig.patch.set_facecolor('white')     # Set the outer colour to white
    ax1 = fig.add_subplot(211,  ylabel='Price in $')
    
    # Plot the AAPL closing price overlaid with the moving averages
    bars['close'].plot(ax=ax1, color='r', lw=2.)
    signals[['short_mavg', 'long_mavg']].plot(ax=ax1, lw=2.)

     # Plot the "buy" trades against AAPL
    ax1.plot(signals.ix[signals.positions == 1.0].index, 
              signals.short_mavg[signals.positions == 1.0],
              '^', markersize=10, color='m')

     # Plot the "sell" trades against AAPL
    ax1.plot(signals.ix[signals.positions == -1.0].index, 
              signals.short_mavg[signals.positions == -1.0],
              'v', markersize=10, color='k')

     # Plot the equity curve in dollars
     ax2 = fig.add_subplot(212, ylabel='Portfolio value in $')
    returns['total'].plot(ax=ax2, lw=2.)

     # Plot the "buy" and "sell" trades against the equity curve
    ax2.plot(returns.ix[signals.positions == 1.0].index, 
              returns.total[signals.positions == 1.0],
              '^', markersize=10, color='m')
     ax2.plot(returns.ix[signals.positions == -1.0].index, 
              returns.total[signals.positions == -1.0],
              'v', markersize=10, color='k')

   # Plot the figure
     fig.show()
   

IndentationError: unexpected indent (<ipython-input-1-b5c435ed9455>, line 53)